In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sampling import fetch_joined_data
df = await fetch_joined_data()
df = df.sort_values(['meeting_timestamp'])

In [3]:
from elastic_search import VectorSearchEngine
search_engine = VectorSearchEngine(device=0)
search_engine.create_index(df)


Using device: cuda:0
Creating field-specific indices...
Processing topic_name...


  0%|          | 0/400 [00:00<?, ?it/s]

Processing summary...


  0%|          | 0/400 [00:00<?, ?it/s]

Processing details...


  0%|          | 0/400 [00:00<?, ?it/s]

In [4]:
meeting_speaker_sets = df.groupby('meeting_id')['speaker_name'].agg(set)
meeting_speaker_sets = meeting_speaker_sets.apply(frozenset)
meeting_combinations = meeting_speaker_sets.value_counts()
print(meeting_combinations.head(20))

speaker_name
(Dmitriy Grankin, Olga Nemirovskaya)                                    33
(Dmitriy Grankin)                                                       32
(nan)                                                                   20
(Dmitriy Grankin, Sergey Ryabenko)                                      15
(nan (0), nan (1))                                                       6
(Dmitriy Grankin, Andrey Pisankin)                                       4
(Nikolay, Dmitriy Grankin, Sergey Ryabenko)                              3
(Dmitriy Grankin, Alex Shevliakov)                                       2
(nan, nan (0), nan (1))                                                  2
(Dmitriy Grankin, Matt Lewis)                                            2
(Dmitriy Grankin, Ilia Semukhin)                                         2
(Alex Shevliakov, Dmitry Grankin)                                        2
(Dmitriy Grankin, Rick Tousseyn)                                         2
(Dmitriy Gra

In [5]:
team = ['Dmitry Grankin','Olga Nemirovskaya','Dmitriy Grankin','Sergey Ryabenko','Alex Shevliakov','Karine Stepanyan','Andrey Pisankin',
        'Okafor Ikenna','nan']

In [6]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [7]:
search_engine.get_search_hints("product ", max_hints=5)

['Product Hunt',
 'Product Hunter',
 'Product School',
 'product hunter',
 'Product Managers']

In [ ]:
from IPython.display import Markdown
from core import system_msg, user_msg, generic_call_stream

In [22]:
from prompts import Prompts
prompts = Prompts()


In [74]:
query = "Lara"
main_results = search_engine.search(
    query=query,
    k=200,
    min_similarity=0.49,
    exact_match_boost=0.3,  # Moderate boost for exact matches
    return_scores=True
)
speaker_results = search_engine.search_by_speaker(
    query,
    k=200,
    min_similarity=0.49,  # Higher threshold for more exact matches
    return_scores=True
)
main_results = main_results[['topic_name','speaker_name','summary','details']]
speaker_results = speaker_results[['topic_name','speaker_name','summary','details']]
results = pd.concat([main_results, speaker_results]).drop_duplicates()
context = results[['topic_name','speaker_name','summary','details']].to_csv(index=False)
report = await generic_call_stream([system_msg(prompts.perplexity+' render as markdown buller points'), 
                              user_msg(f"Query: {query}\nContext: {context}")])


Markdown(report)

Lara is involved in various aspects of influencer marketing and content creation, as highlighted by Olga Nemirovskaya. She coordinates with influencers to publish promotional posts for Vexy and is currently negotiating with two influencers to secure content publication within two weeks[1][6]. Additionally, Lara is engaged in discussions regarding workshop formats and is expected to provide contracts necessary for project progression[2][5]. 

Lara Vargas, another individual mentioned, is an influencer marketing professional from Brazil, emphasizing the importance of connecting brands with relevant audiences in Brazil. She has experience in various sectors and is working on strategies to enhance visibility through influencer collaborations[3][4]. 

Overall, Lara's contributions are crucial for establishing a strong influencer marketing strategy and ensuring effective content dissemination across social media platforms.


Lara is involved in various aspects of influencer marketing and content creation, as highlighted by Olga Nemirovskaya. She coordinates with influencers to publish promotional posts for Vexy and is currently negotiating with two influencers to secure content publication within two weeks[1][6]. Additionally, Lara is engaged in discussions regarding workshop formats and is expected to provide contracts necessary for project progression[2][5]. 

Lara Vargas, another individual mentioned, is an influencer marketing professional from Brazil, emphasizing the importance of connecting brands with relevant audiences in Brazil. She has experience in various sectors and is working on strategies to enhance visibility through influencer collaborations[3][4]. 

Overall, Lara's contributions are crucial for establishing a strong influencer marketing strategy and ensuring effective content dissemination across social media platforms.

In [59]:
pd.set_option('display.max_colwidth', 100)

In [60]:
results

,summary_index,summary,details,referenced_text,topic_name,topic_type,meeting_id,meeting_timestamp,speaker_name,other_speakers,similarity_score
1372,6,Planned to analyze competitors' marketing strategies to inform future campaigns.,"Olga and Dmitriy discussed the importance of understanding competitors' successful strategies, p...","Olga Nemirovskaya: И плюс к этому, я сегодня еще много смотрела тоже по конкурентам, что они де...",Analyze competitors' marketing strategies,task,99ade764-fd60-4e9a-9621-66cc037cb865,2024-08-27 19:34:48.996,Olga Nemirovskaya,[Dmitriy Grankin],1.230171
2096,4,"Dmitrii Bashkirov and Dmitriy Grankin discussed and identified key competitors to analyze, focus...","Competitors selected for analysis include Fatom, TLDV, FireFiles, Otter, MidGeek, and SuperPower...","Dmitrii Bashkirov: Взял я вот, посмотрел, я взял Fatom, TLDV, FireFiles, Otter и еще взял MidGe...",Identify key competitors for analysis,task,ae547dc7-22b0-4628-a715-4d0377adfc6c,2024-09-04 11:23:50.070,Dmitriy Grankin,"[Dmitrii Bashkirov, Dmitrii Grankin]",1.210878
2618,3,Dmitriy discussed the need to understand user frustrations with competitors to better position t...,Dmitriy highlighted that users often express dissatisfaction with bots and the quality of summar...,"Dmitriy Grankin: Я вчера пообщался с пользователями, и одно и то же все говорят. | Dmitriy Gran...",Identify user pain points with competitors,task,d7d8d9e4-f349-42b3-9599-f9707f1405f8,2024-09-10 09:28:46.960,Dmitriy Grankin,[Olga Nemirovskaya],1.173120
1374,9,Discussed creating video content inspired by successful competitors' strategies.,Olga mentioned analyzing successful videos from competitors to replicate their success. This tas...,Dmitriy Grankin: августа 24 года отметила что необходимо договоренности были достигнуты контент...,Create engaging video content based on competitors' success,task,99ade764-fd60-4e9a-9621-66cc037cb865,2024-08-27 19:34:48.996,Olga Nemirovskaya,[Dmitriy Grankin],1.161271
2615,0,Olga reviewed the Google Ads usage of competitors and found that most do not utilize it effectiv...,"Olga discovered that competitors like Fireflies, Notte, and Supernormal use Google Ads minimally...","Olga Nemirovskaya: Слушай, ну вот я, видишь, вчера как бы долго просидела в Семраше, смотрела с...",Analyze Google Ads usage among competitors,task,d7d8d9e4-f349-42b3-9599-f9707f1405f8,2024-09-10 09:28:46.960,Olga Nemirovskaya,[Dmitriy Grankin],1.128424
...,...,...,...,...,...,...,...,...,...,...,...
2396,0,Discuss the competitor's SEO marketing strategies that were shared previously and how they can b...,Olga reviewed the competitor's SEO marketing strategies and has integrated some elements into Go...,Dmitry Grankin: хотим поговорить он сделал competitor seo маркетинг он сделал в | Olga Nemirovs...,Review competitor SEO marketing strategies,task,55fb40e1-096c-4280-b205-aa4cc68f6c76,2024-09-06 14:32:06.780,Dmitry Grankin,[Olga Nemirovskaya],0.506503
526,26,AI transcription refers to the use of artificial intelligence to convert speech into text.,"Dmitry's product utilizes AI for transcription and summarization, aiming to differentiate from c...","Dmitry Grankin: The last idea I tried out, I was approaching people, reaching people from G2, w...",AI Transcription,concept,3540f190-1f83-4100-8d6f-e648752f336d,2024-09-26 12:02:28.836,Dmitry Grankin,[Andy Black],0.505449
1229,4,Dmitry and Özay discussed the crowded market for their products and the strategies needed to dif...,Dmitry acknowledged the presence of competition but argued that the market is still sparse. He e...,"Özay Demirezen: But how did you decide to enter this crowded market? Like, uh, This market is c...",Market competition and product differentiation,discussion,50d5d659-3a57-42f2-8f32-31c7145ebe91,2024-10-24 09:32:32.980,Dmitry Grankin,[Özay Demirezen],0.504105
876,5,The team is addressing inquiries regarding GDPR compliance in their outreach efforts.,Dmitriy mentioned that they are fully compliant with GDPR 

In [65]:

results

,speaker_name,topic_name,summary,details,meeting_timestamp,speaker_similarity
2153,Shota Arabuli,project management,"Project management involves planning, executing, and overseeing projects to achieve specific goa...",Shota shares his approach to managing multiple projects and the role of AI in streamlining proje...,2024-09-04 15:00:03.180,0.658235
2168,Shota Arabuli,business development,Business development involves identifying growth opportunities and building relationships to dri...,Shota discusses the role of business development in validating product ideas and engaging potent...,2024-09-04 15:00:03.180,0.658235
2158,Shota Arabuli,data analysis,"Data analysis is the process of inspecting, cleansing, and modeling data to discover useful info...",Shota discusses the role of data analysis in validating business hypotheses and informing produc...,2024-09-04 15:00:03.180,0.658235
2167,Shota Arabuli,customer needs,Customer needs are the requirements and expectations of users that drive product development and...,The meeting focuses on understanding and addressing customer needs through effective communicati...,2024-09-04 15:00:03.180,0.658235
2150,Shota Arabuli,business model,"A business model outlines how a company creates, delivers, and captures value, particularly in t...",Shota discusses the importance of validating business models through customer feedback and itera...,2024-09-04 15:00:03.180,0.658235
2159,Shota Arabuli,market research,Market research involves gathering and analyzing information about consumers and competitors to ...,Shota mentions the significance of market research in understanding customer needs and validatin...,2024-09-04 15:00:03.180,0.658235
2147,Shota Arabuli,AI,"AI refers to artificial intelligence technologies that can assist in various business processes,...","The discussion revolves around the use of AI in optimizing workflows, enhancing communication, a...",2024-09-04 15:00:03.180,0.658235
2140,Shota Arabuli,Effective communication during interviews,"The importance of clear communication and tailored questioning in interviews was highlighted, wi...",Participants discussed the need for flexibility in questioning and the ability to adapt based on...,2024-09-04 15:00:03.180,0.658235
2142,Shota Arabuli,Shota Arabuli,"Shota Arabuli is a participant in the meeting, involved in multiple early-stage projects and dis...",Shota shares insights on using AI for project management and emphasizes the importance of respon...,2024-09-04 15:00:03.180,0.658235
2179,Shota Arabuli,AI assistance in user interviews,"The potential for AI to assist in user interviews and generate relevant questions was discussed,...","AI could help interviewers by suggesting questions based on the context of the conversation, mak...",2024-09-04 16:04:02.020,0.658235
